### LCEL - LangChain Expression Language

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

TOGETHER_API_KEY = "41bad9b287c02b34b4a844bcfc21f18225cc0852f983652c8952908795a75c07"

openai.api_key = TOGETHER_API_KEY

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Cleans the response to a simple string
from langchain.schema.output_parser import StrOutputParser

### Chain

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)

model = ChatOpenAI(
    openai_api_base="https://api.together.xyz/v1",
    openai_api_key=TOGETHER_API_KEY,
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
)

output_parser = StrOutputParser()

In [ ]:
# Simple chain: LCEL

chain = prompt | model | output_parser

In [5]:
chain.invoke({"topic": "bears"})

" Why don't bears like to play hide and seek?\nBecause they always get spotted!"

* Complex Chaining

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

# Vector Stores
from langchain.vectorstores import DocArrayInMemorySearch

In [7]:
from together import Together

client = Together(api_key=TOGETHER_API_KEY)

In [ ]:
from langchain_core.embeddings import Embeddings
import numpy as np

# we make a class for the TogetherAI embedding (instead of OpenAI's embedding)
class TogetherAIEmbeddings(Embeddings):
    
    """Custom langchain embedding class for TogetherAI"""
    
    def __init__(
        self,
        model="togethercomputer/m2-bert-80M-8k-retrieval", # Embedding model
        api_key=None
    ):
        self.client = Together(api_key=api_key)
        self.model = model

    # Embedding a full document    
    def embed_documents(self, texts):
        # embed multiple documents
        response = self.client.embeddings.create(model=self.model, input=texts)
        return [x.embedding for x in response.data]

    # Embedding a single query
    def embed_query(self, text):
        """Embeds a single query"""
        response = self.client.embeddings.create(model=self.model, input=[text])  # pass as list of inputs
        return response.data[0].embedding # Extract the first embedding

In [9]:
response = client.embeddings.create(
  model = "togethercomputer/m2-bert-80M-8k-retrieval",
  input = [
    "Our solar system orbits the Milky Way galaxy at about 515,000 mph",
    "Jupiter's Great Red Spot is a storm that has been raging for at least 350 years."
  ]
)

In [10]:
len([x.embedding for x in response.data][0])

768

In [11]:
embedding = TogetherAIEmbeddings(api_key=TOGETHER_API_KEY)

**Vectorstore with OSS Embeddings**

In [ ]:

vectorstore = DocArrayInMemorySearch.from_texts(
    texts=["harrison worked at kensho", "bears like to eat honey"], # Content
    embedding=TogetherAIEmbeddings(api_key=TOGETHER_API_KEY)  # Embedding model
)

# Create a retriever from the vector store
retriever = vectorstore.as_retriever()

/home/jerlshin/anaconda3/envs/gnn/lib/python3.9/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [13]:
query = "Where did Harrison work?"
docs = retriever.invoke(query)

In [14]:
docs

[Document(metadata={}, page_content='harrison worked at kensho'),
 Document(metadata={}, page_content='bears like to eat honey')]

In [15]:
retriever.get_relevant_documents("what do bears like to eat")

/tmp/ipykernel_20631/2656275837.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("what do bears like to eat")


[Document(metadata={}, page_content='bears like to eat honey'),
 Document(metadata={}, page_content='harrison worked at kensho')]

In [16]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
# Data preprocessor. It runs multiple functions in parallel to prepare the data for the next step in the chain
from langchain.schema.runnable import RunnableMap 

# Mapping inputs to Runnables

In [ ]:
chain = RunnableMap(
    # lambda function mapping
    {
        "context": lambda x: retriever.get_relevant_documents(x["question"]), # fetches the relevant text from the vector db
        "question": lambda x: x["question"] # original string
    }
) | prompt | model | output_parser

In [19]:
chain.invoke({"question": "Where di harrison work?"})

' Based on the provided context, Harrison works at Kensho.'

In [20]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [21]:
inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(metadata={}, page_content='harrison worked at kensho'),
  Document(metadata={}, page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

### Function Binding

* Process of telling the LLM the list of tools it is allowed to use for user's request

In [1]:
functions = [
    {
        "name": "weather_search",
        "description": "Search for weather given an airport code",
        
        "parameters": {
            "type": "object",
            
            "properties": {
                "airport_code": {
                    "type": "string",
                    "description": "The airport code to get the weather for"
                },
            },
            
            "required": ["airport_code"]
        }
    }
]

In [23]:
prompt = ChatPromptTemplate.from_messages(
    [("human", "{input}")]
)

In [ ]:
model = ChatOpenAI(
    openai_api_base="https://api.together.xyz/v1",
    openai_api_key=TOGETHER_API_KEY,
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
).bind(functions=functions) # Attaching these function definition to every request sent to the AI


In [ ]:
chain = (
    # User input and prepares it with RunnableMap
    RunnableMap({"input": lambda x: x["input"]})  # Pass question as input
    | prompt
    | model
    | StrOutputParser()
)

In [36]:
query = {"input": "what is the weather in sf"}

response = chain.invoke(query)

In [37]:
response

''

In [29]:
response = model.invoke("Extract the airport code from: 'What is the weather in SF?'")
print(response)


content='' additional_kwargs={'tool_calls': [{'id': 'call_er1iv94b7wg9cbw34t8mxqpl', 'function': {'arguments': '{"airport_code":"SF"}', 'name': 'weather_search'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 488, 'total_tokens': 523, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'mistralai/Mistral-7B-Instruct-v0.1', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run-2a09afd5-d294-4cd5-bee0-ae99a37305ea-0' tool_calls=[{'name': 'weather_search', 'args': {'airport_code': 'SF'}, 'id': 'call_er1iv94b7wg9cbw34t8mxqpl', 'type': 'tool_call'}] usage_metadata={'input_tokens': 488, 'output_tokens': 35, 'total_tokens': 523, 'input_token_details': {}, 'output_token_details': {}}


In [28]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [113]:
model = model.bind(functions=functions)

In [114]:
runnable = prompt | model

In [115]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content=" I'm sorry, but I don't have information on recent sport events. My current capabilities are limited to searching for weather based on an airport code.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 848, 'total_tokens': 881, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'mistralai/Mistral-7B-Instruct-v0.1', 'system_fingerprint': None, 'finish_reason': 'eos', 'logprobs': None}, id='run-0eb50a49-adfc-456c-b462-e4e2d7aaf37f-0', usage_metadata={'input_tokens': 848, 'output_tokens': 33, 'total_tokens': 881, 'input_token_details': {}, 'output_token_details': {}})